In [ ]:
# Import Packages
import csv
import pandas as pd
import numpy as np
import subprocess
import google.colab # if colab exists, install pysqlite-binary
subprocess.run(['pip', 'install', 'pysqlite3-binary'], capture_output=False)
import pysqlite3 as sqlite3
import matplotlib.pyplot as plt

In [ ]:
# Read Data and create DataBase
df_1 = pd.read_csv("/content/TCONFIRM_20230614.csv")

In [ ]:
conn = sqlite3.connect('LionFund.db')  # Replace 'your_database.db' with your database file
cursor = conn.cursor()

In [ ]:
cursor.execute('CREATE TABLE IF NOT EXISTS Lion_Fund (Date_time TIME, User_id INTEGER, Product_id INTEGER)')

In [ ]:
with open('/content/TCONFIRM_20230614.csv', 'r') as file:  # Replace 'your_file.csv' with your CSV file name
    csv_data = csv.reader(file)
    next(csv_data)  # Skip the header row if it exists
    cursor.executemany('INSERT INTO Lion_Fund(Date_time, User_id, Product_id) VALUES (:1, :2, :3)', csv_data)  # Replace with your table name

In [ ]:
pd.read_sql_query("select name from sqlite_master where type='table' order by name",conn)

<ipython-input-6-79e31349b8b1>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select name from sqlite_master where type='table' order by name",conn)


,name
0,Lion_Fund


In [ ]:
pd.read_sql_query("select * from Lion_Fund",conn)

<ipython-input-7-c436a9a14423>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from Lion_Fund",conn)


,Date_time,User_id,Product_id
0,2023-01-03 00:00:00.000,39500000015571,320007
1,2023-01-03 00:00:00.000,7830019642211,320007
2,2023-01-03 00:00:00.000,10030017623818,320007
3,2023-01-03 00:00:00.000,6730014589855,320007
4,2023-01-03 00:00:00.000,2230021523587,320007
...,...,...,...
12085,2023-02-28 00:00:00.000,10630003698586,10356
12086,2023-02-28 00:00:00.000,36900000222326,163209
12087,2023-02-28 00:00:00.000,36900000024132,163208
12088,2023-02-28 00:00:00.000,4830001891961,10356


In [ ]:
# TEST
df_1[df_1['User_id']==40900000215056]

,Date_time,User_id,Product_id
11295,2023-02-27 00:00:00.000,40900000215056,320006
11296,2023-02-27 00:00:00.000,40900000215056,320007
11491,2023-02-27 00:00:00.000,40900000215056,1743
11946,2023-02-28 00:00:00.000,40900000215056,320013


In [ ]:

# APRIORI

In [ ]:


df_1 = pd.read_sql_query("select * from Lion_Fund",conn)
#df_concat = df_1.groupby('C_CUST_ID')['TO_CHAR(C_FUNDCODE)'].apply(','.join).reset_index()
df_1

<ipython-input-10-2cd078d33337>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_1 = pd.read_sql_query("select * from Lion_Fund",conn)


,Date_time,User_id,Product_id
0,2023-01-03 00:00:00.000,39500000015571,320007
1,2023-01-03 00:00:00.000,7830019642211,320007
2,2023-01-03 00:00:00.000,10030017623818,320007
3,2023-01-03 00:00:00.000,6730014589855,320007
4,2023-01-03 00:00:00.000,2230021523587,320007
...,...,...,...
12085,2023-02-28 00:00:00.000,10630003698586,10356
12086,2023-02-28 00:00:00.000,36900000222326,163209
12087,2023-02-28 00:00:00.000,36900000024132,163208
12088,2023-02-28 00:00:00.000,4830001891961,10356


In [ ]:
df_2 = df_1.groupby('User_id').agg({'Product_id': lambda x: x.tolist()},axis=1).reset_index()
df_2

,User_id,Product_id
0,1000030401658,"[320007, 320007, 320007, 320007, 320007, 320007]"
1,1000030799041,[1706]
2,1000030806861,[163208]
3,1000030880223,[1706]
4,1000030889902,[163209]
...,...,...
5075,40900000205074,[320007]
5076,40900000208969,"[10349, 10349, 10349, 10349]"
5077,40900000209061,[1208]
5078,40900000212814,[1208]


In [ ]:
# install KiwiApriori
subprocess.run(['pip', 'install', 'kiwi-apriori'], capture_output=False)


CompletedProcess(args=['pip', 'install', 'kiwi-apriori'], returncode=0)

In [ ]:
import apriori

# Total of 5080 Users, Min_support = 0.00059 equals to 3 appearences, min_lift = 1 means that they are all positively correlated


for rule in apriori.generate_rules(df_2['Product_id'], min_support=0.00059, min_lift = 1):

    msg = (f'{rule.format_rule():20s}\t\t'
           f'(support={rule.support:0.4f}, confidence={rule.confidence:0.4f}, lift={rule.lift:0.4f})')
    print(msg)

{320020} ---> {320022}		(support=0.0006, confidence=0.0204, lift=4.1469)
{320022} ---> {320020}		(support=0.0006, confidence=0.1200, lift=4.1469)
{320003} ---> {320020}		(support=0.0010, confidence=0.0962, lift=3.3229)
{320020} ---> {320003}		(support=0.0010, confidence=0.0340, lift=3.3229)
{1744} ---> {320020}		(support=0.0008, confidence=0.0816, lift=2.8210)
{320020} ---> {1744}		(support=0.0008, confidence=0.0272, lift=2.8210)
{1744} ---> {1743}  		(support=0.0006, confidence=0.0612, lift=3.7026)
{1743} ---> {1744}  		(support=0.0006, confidence=0.0357, lift=3.7026)
{1706} ---> {1411}  		(support=0.0006, confidence=0.0588, lift=2.7927)
{1411} ---> {1706}  		(support=0.0006, confidence=0.0280, lift=2.7927)
{2560} ---> {1411}  		(support=0.0012, confidence=0.0337, lift=1.6003)
{1411} ---> {2560}  		(support=0.0012, confidence=0.0561, lift=1.6003)
{2051} ---> {1411}  		(support=0.0008, confidence=0.0230, lift=1.0914)
{1411} ---> {2051}  		(support=0.0008, confidence=0.0374, lift=1.0914

In [ ]:
rules = apriori.generate_rules(df_2['Product_id'], min_support=0.00059, min_lift = 1)

df_rules = pd.DataFrame(rules,columns = ['Product_base','Product_lead','support','confidence','lift'])
df_rules

,Product_base,Product_lead,support,confidence,lift
0,"(base, {320020})","(leads_to, {320022})","(support, 0.0005905511811023622)","(confidence, 0.02040816326530612)","(lift, 4.146938775510204)"
1,"(base, {320022})","(leads_to, {320020})","(support, 0.0005905511811023622)","(confidence, 0.12)","(lift, 4.146938775510204)"
2,"(base, {320003})","(leads_to, {320020})","(support, 0.000984251968503937)","(confidence, 0.09615384615384616)","(lift, 3.3228676085818942)"
3,"(base, {320020})","(leads_to, {320003})","(support, 0.000984251968503937)","(confidence, 0.03401360544217687)","(lift, 3.322867608581894)"
4,"(base, {1744})","(leads_to, {320020})","(support, 0.0007874015748031496)","(confidence, 0.08163265306122448)","(lift, 2.821046786061363)"
5,"(base, {320020})","(leads_to, {1744})","(support, 0.0007874015748031496)","(confidence, 0.027210884353741496)","(lift, 2.821046786061363)"
6,"(base, {1744})","(leads_to, {1743})","(support, 0.0005905511811023622)","(confidence, 0.061224489795918366)","(lift, 3.7026239067055395)"
7,"(base, {1743})","(leads_to, {1744})","(support, 0.0005905511811023622)","(confidence, 0.03571428571428571)","(lift, 3.702623906705539)"
8,"(base, {1706})","(leads_to, {1411})","(support, 0.0005905511811023622)","(confidence, 0.058823529411764705)","(lift, 2.792743265530511)"
9,"(base, {1411})","(leads_to, {1706})","(support, 0.0005905511811023622)","(confidence, 0.02803738317757009)","(lift, 2.792743265530511)"


In [ ]:
# Clean the DataFrame

df_rules['Product_base']=df_rules['Product_base'].apply(lambda x: x[1])
df_rules['Product_lead']=df_rules['Product_lead'].apply(lambda x: x[1])
df_rules['support']=df_rules['support'].apply(lambda x: x[1])
df_rules['confidence']=df_rules['confidence'].apply(lambda x: x[1])
df_rules['lift']=df_rules['lift'].apply(lambda x: x[1])
df_rules

,Product_base,Product_lead,support,confidence,lift
0,{320020},{320022},0.000591,0.020408,4.146939
1,{320022},{320020},0.000591,0.120000,4.146939
2,{320003},{320020},0.000984,0.096154,3.322868
3,{320020},{320003},0.000984,0.034014,3.322868
4,{1744},{320020},0.000787,0.081633,2.821047
5,{320020},{1744},0.000787,0.027211,2.821047
6,{1744},{1743},0.000591,0.061224,3.702624
7,{1743},{1744},0.000591,0.035714,3.702624
8,{1706},{1411},0.000591,0.058824,2.792743
9,{1411},{1706},0.000591,0.028037,2.792743


In [ ]:
# Sort by values support desc, confidence desc, lift desc
df_rules.sort_values(by = ['support','confidence','lift'],ascending = False)

,Product_base,Product_lead,support,confidence,lift
31,{320020},{1208},0.003543,0.122449,1.416949
30,{1208},{320020},0.003543,0.041002,1.416949
29,{1743},{320020},0.001181,0.071429,2.468416
11,{1411},{2560},0.001181,0.056075,1.600336
28,{320020},{1743},0.001181,0.040816,2.468416
10,{2560},{1411},0.001181,0.033708,1.600336
33,{1706},{2560},0.000984,0.098039,2.797973
2,{320003},{320020},0.000984,0.096154,3.322868
25,{10349},{320020},0.000984,0.055556,1.919879
3,{320020},{320003},0.000984,0.034014,3.322868


In [ ]:
# Filter into duo-package and trio-package

In [ ]:
# Duo-package
Duo_packs = df_rules[(df_rules['Product_base'].apply(lambda x: len(x))==1) & (df_rules['Product_lead'].apply(lambda x: len(x))==1)]
Duo_packs.sort_values(by = ['support','confidence','lift'],ascending = False,inplace = True)
Duo_packs

<ipython-input-19-4bf5351e8bc0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Duo_packs.sort_values(by = ['support','confidence','lift'],ascending = False,inplace = True)


,Product_base,Product_lead,support,confidence,lift
31,{320020},{1208},0.003543,0.122449,1.416949
30,{1208},{320020},0.003543,0.041002,1.416949
29,{1743},{320020},0.001181,0.071429,2.468416
11,{1411},{2560},0.001181,0.056075,1.600336
28,{320020},{1743},0.001181,0.040816,2.468416
10,{2560},{1411},0.001181,0.033708,1.600336
33,{1706},{2560},0.000984,0.098039,2.797973
2,{320003},{320020},0.000984,0.096154,3.322868
25,{10349},{320020},0.000984,0.055556,1.919879
3,{320020},{320003},0.000984,0.034014,3.322868


In [ ]:
# Trio Packages
Trio_packs = df_rules[df_rules['Product_base'].apply(lambda x: len(x))+df_rules['Product_lead'].apply(lambda x: len(x))==3]
Trio_packs.sort_values(by = ['support','confidence','lift'],ascending = False,inplace = True)
Trio_packs

<ipython-input-20-99c354646d93>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Trio_packs.sort_values(by = ['support','confidence','lift'],ascending = False,inplace = True)


,Product_base,Product_lead,support,confidence,lift
44,"{10349, 1208}",{320020},0.000591,0.600000,20.734694
45,"{320020, 10349}",{1208},0.000591,0.600000,6.943052
43,"{320020, 1208}",{10349},0.000591,0.166667,9.407407
38,"{320007, 2560}",{1411},0.000591,0.136364,6.474087
39,"{320007, 1411}",{2560},0.000591,0.115385,3.292999
42,{10349},"{320020, 1208}",0.000591,0.033333,9.407407
37,{1411},"{320007, 2560}",0.000591,0.028037,6.474087
41,{320020},"{10349, 1208}",0.000591,0.020408,20.734694
36,{2560},"{320007, 1411}",0.000591,0.016854,3.292999
40,{1208},"{320020, 10349}",0.000591,0.006834,6.943052


In [ ]:
# Usings sequence Mining from pyMining that takes orders into account

In [ ]:
# Install pyMining
subprocess.run(['pip', 'install', 'pymining'], capture_output=False)

CompletedProcess(args=['pip', 'install', 'pymining'], returncode=0)

In [ ]:
from pymining import seqmining


# Run the PrefixSpan algorithm
frequent_patterns = seqmining.freq_seq_enum(df_2['Product_id'],min_support = 3)
df_seqmining = pd.DataFrame(frequent_patterns)
# Print the frequent patterns with combination of 2 funds
for pattern, support in frequent_patterns:
  if len(pattern)==2:
    print(pattern, support)


(2560, 320007) 18
(1411, 1411) 49
(320004, 320007) 3
(2051, 1706) 4
(1780, 1780) 7
(1744, 2051) 3
(10356, 10356) 28
(538, 538) 12
(320010, 320010) 4
(1411, 2051) 4
(6429, 320007) 3
(320006, 320006) 35
(2560, 1411) 6
(2051, 1744) 3
(320018, 320018) 10
(320004, 320004) 17
(10349, 320020) 5
(1743, 1208) 3
(1411, 1706) 3
(320021, 320021) 17
(1744, 320020) 3
(320007, 1411) 17
(971, 971) 20
(320020, 1208) 11
(320006, 320007) 5
(320015, 320015) 4
(1743, 320020) 4
(320003, 320003) 19
(320002, 320002) 9
(10349, 10349) 38
(320007, 320020) 3
(6008, 6008) 4
(1411, 320007) 21
(971, 1208) 3
(320020, 320003) 3
(2051, 2051) 83
(1351, 1351) 8
(1208, 1208) 96
(320007, 320006) 5
(10349, 1208) 3
(2137, 2137) 3
(320011, 320011) 6
(320020, 320007) 4
(320007, 2560) 17
(1208, 1744) 3
(320020, 320020) 40
(320020, 1411) 3
(320020, 1743) 6
(1706, 1706) 18
(2291, 2291) 4
(2560, 1706) 5
(1743, 1743) 36
(1208, 320020) 11
(2051, 2560) 4
(163208, 163208) 51
(320007, 538) 6
(320003, 320020) 5
(14551, 14551) 4
(538, 32

In [ ]:
# Create a DataFrame
df_seqmining = pd.DataFrame(frequent_patterns,columns = ['Product_combinations','counts'])
df_seqmining

,Product_combinations,counts
0,"(320004, 320004, 320004, 320004)",3
1,"(1743, 1743, 1743, 1743, 1743)",11
2,"(320007, 320007, 320007, 2051, 2051, 320007, 3...",4
3,"(1411, 320007, 320007, 1411, 1411, 320007, 320...",3
4,"(320007, 2560, 2560, 2560)",7
...,...,...
796,"(320007, 320007, 2051, 2051, 320007, 320007, 2...",3
797,"(320022, 320022)",8
798,"(2560, 2560)",76
799,"(2560, 2560, 2560, 2560, 2560, 2560, 2560, 256...",6


In [ ]:
# Filter into duo_packs
def set_not_equal_duo(a):
  if a[0] != a[1]:
    return True
  return False
# Create a duo pack DataFrame
df_seqmining_duo = df_seqmining[df_seqmining['Product_combinations'].apply(lambda x: len(x)==2)]
# Sort the values by counts desc and filter out combinations with two replicated funds
df_seqmining_duo = df_seqmining_duo[df_seqmining_duo['Product_combinations'].apply(set_not_equal_duo)].sort_values(by='counts',ascending = False).reset_index()
df_seqmining_duo

,index,Product_combinations,counts
0,763,"(320007, 2051)",22
1,385,"(1411, 320007)",21
2,708,"(2051, 320007)",19
3,56,"(2560, 320007)",18
4,473,"(320007, 2560)",17
5,255,"(320007, 1411)",17
6,534,"(1208, 320020)",11
7,293,"(320020, 1208)",11
8,553,"(320007, 538)",6
9,174,"(2560, 1411)",6


In [ ]:
# Filter into Trio_packs
def set_not_equal_trio(a):
  if a[0] != a[1] or a[0]!= a[2]:
    return True
  return False
# Create a trio pack DataFrame
df_seqmining_trio = df_seqmining[df_seqmining['Product_combinations'].apply(lambda x: len(x)==3)]
# Sort the values by counts desc and filter out combinations with two replicated funds
df_seqmining_trio = df_seqmining_trio[df_seqmining_trio['Product_combinations'].apply(set_not_equal_trio)].sort_values(by='counts',ascending = False).reset_index()
df_seqmining_trio

,index,Product_combinations,counts
0,575,"(320007, 2051, 320007)",13
1,600,"(2051, 320007, 320007)",13
2,77,"(320007, 320007, 2051)",13
3,120,"(2051, 320007, 2051)",12
4,694,"(2051, 2051, 320007)",12
5,167,"(320007, 2051, 2051)",12
6,772,"(320007, 1411, 320007)",12
7,351,"(2560, 320007, 2560)",11
8,713,"(2560, 320007, 320007)",10
9,213,"(320007, 320007, 2560)",10


In [ ]:
# Recommendation Based on one Fund
def Fund_Recommendation(Fund_code):
  df_seqmining_tmp = pd.concat([df_seqmining_duo, df_seqmining_trio], axis=0)
  df_tmp = df_seqmining_tmp[df_seqmining_tmp['Product_combinations'].apply(lambda x: True if x[0]== Fund_code else False)]
  list_tmp = list(df_tmp['Product_combinations'])
  recommendation_list = list(zip([ x[1:] for x in list_tmp],df_tmp['counts']))
  print(f'Based on your purchase of Fund {Fund_code}\nWe recommend these Funds below:')
  for i, fund in enumerate(recommendation_list):
    print(f'{i+1}) Fund code: {fund[0]}, Fund count: {fund[1]}')


In [ ]:
Fund_Recommendation(320007)

Based on your purchase of Fund 320007
We recommend these Funds below:
1) Fund code: (2051,), Fund count: 22
2) Fund code: (2560,), Fund count: 17
3) Fund code: (1411,), Fund count: 17
4) Fund code: (538,), Fund count: 6
5) Fund code: (320006,), Fund count: 5
6) Fund code: (320020,), Fund count: 3
7) Fund code: (2051, 320007), Fund count: 13
8) Fund code: (320007, 2051), Fund count: 13
9) Fund code: (2051, 2051), Fund count: 12
10) Fund code: (1411, 320007), Fund count: 12
11) Fund code: (320007, 2560), Fund count: 10
12) Fund code: (1411, 1411), Fund count: 9
13) Fund code: (320007, 1411), Fund count: 9
14) Fund code: (2560, 320007), Fund count: 9
15) Fund code: (2560, 2560), Fund count: 8
16) Fund code: (538, 320007), Fund count: 3


In [ ]:
# One Fund Recommendation Based on one Fund
def Fund_Recommendation_1_to_1(Fund_code):
  #df_seqmining_tmp = pd.concat([df_seqmining_duo, df_seqmining_trio], axis=0)
  df_tmp = df_seqmining_duo[df_seqmining_duo['Product_combinations'].apply(lambda x: True if x[0]== Fund_code else False)]
  list_tmp = list(df_tmp['Product_combinations'])
  recommendation_list = list(zip([ x[1] for x in list_tmp],df_tmp['counts']))
  print(f'Based on your purchase of Fund {Fund_code}\nWe recommend these Funds below:')
  for i, fund in enumerate(recommendation_list):
    print(f'{i+1}) Fund code: {fund[0]}, Fund count: {fund[1]}')

In [ ]:
# Test
Fund_Recommendation_1_to_1(320007)

Based on your purchase of Fund 320007
We recommend these Funds below:
1) Fund code: 2051, Fund count: 22
2) Fund code: 2560, Fund count: 17
3) Fund code: 1411, Fund count: 17
4) Fund code: 538, Fund count: 6
5) Fund code: 320006, Fund count: 5
6) Fund code: 320020, Fund count: 3


In [ ]:
# Two Fund Recommendation Based on one Fund
def Fund_Recommendation_1_to_2(Fund_code):
  #df_seqmining_tmp = pd.concat([df_seqmining_duo, df_seqmining_trio], axis=0)
  df_tmp = df_seqmining_trio[df_seqmining_trio['Product_combinations'].apply(lambda x: True if x[0]== Fund_code else False)]
  list_tmp = list(df_tmp['Product_combinations'])
  recommendation_list = list(zip([ x[1:] for x in list_tmp],df_tmp['counts']))
  print(f'Based on your purchase of Fund {Fund_code}\nWe recommend these Fund combos below:')
  for i, fund in enumerate(recommendation_list):
    print(f'{i+1}) Fund code: {fund[0]}, Fund count: {fund[1]}')

In [ ]:
# Test
Fund_Recommendation_1_to_2(320007)

Based on your purchase of Fund 320007
We recommend these Fund combos below:
1) Fund code: (2051, 320007), Fund count: 13
2) Fund code: (320007, 2051), Fund count: 13
3) Fund code: (2051, 2051), Fund count: 12
4) Fund code: (1411, 320007), Fund count: 12
5) Fund code: (320007, 2560), Fund count: 10
6) Fund code: (1411, 1411), Fund count: 9
7) Fund code: (320007, 1411), Fund count: 9
8) Fund code: (2560, 320007), Fund count: 9
9) Fund code: (2560, 2560), Fund count: 8
10) Fund code: (538, 320007), Fund count: 3


In [ ]:
##########################################################################################################################